### 문제 4

지금까지 만든 3개의 DataFrame을 국가 이름(`Country`)을 기준으로 합쳐서(`merge`) 하나의 DataFrame을 만들어라. 단, `ScimEn_df`에서 상위 15개의 국가만을(Rank 1-15) 포함하라. 만들어진 DataFrame의 인덱스를 국가명(`Country`)으로 설정하라.

결과적으로 만들어진 DataFrame은 국가명(`Country`)을 인덱스로 가지며, 15개의 행을 가지며, 다음의 컬럼들 만을 포함해야 한다.
['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020'].

이런 일을 하나의 함수 `answer_four()`로 구현하라. `answer_four`에서는 위의 문제들에서 작성한 함수들(`answer_one`, `answer_two`, `answer_three`)를 이용한다. 함수 `answer_four`는 결과적으로 국가명을 인덱스로 가지고 15개의 행과 20개의 컬럼을 가진 하나의 DataFrame을 반환한다. 이 DataFrame은 `Rank`에 의해서 정렬되어 있어야 한다.

---
### 풀이

이전에 만든 함수들

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def answer_one():
    df = pd.read_excel('Energy Indicators2017.xlsx', sheet_name=1, nrows=217, header=0)#두번째 sheet에서 218번째 행까지 불러오기
    df.drop(df.iloc[:,[0, 5, 6]], axis=1, inplace=True)#비어있는 열 없애기
    df.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    df['Energy Supply'] = df['Energy Supply']*1000000
    df.loc[df['% Renewable']=='…', '% Renewable'] = np.NaN
    df['% Renewable'] = df['% Renewable'].astype('float')
    df.loc[df['Country']=='Korea, Republic of', 'Country'] = 'South Korea'
    df.loc[df['Country']=='United States of America', 'Country'] = 'United States'
    df.loc[df['Country']=='United Kingdom of Great Britain and Northern Ireland', 'Country'] = 'United Kingdom'
    df.loc[df['Country']=='China, Hong Kong SAR', 'Country'] = 'Hong Kong'
    country_val = df['Country'].values
    for i in range(len(country_val)):
        country_val[i] = re.sub(r' \([^)]*\)', '', country_val[i])
    df['Country'] = country_val
    return df

In [3]:
def answer_two():
    path = './world_bank2020.csv'
    GDP_df = pd.read_csv(path, skiprows=4)
    GDP_df = GDP_df.filter(regex='(Country Name|20(1[1-9]|20))', axis=1)
    GDP_df = GDP_df.rename(columns={"Country Name": "Country"})
    GDP_df = GDP_df.set_index("Country")
    GDP_df = GDP_df.rename(index= {"Korea, Rep.":"South Korea","Iran, Islamic Rep.":"Iran", 
                                   "Hong Kong SAR, China": "Hong Kong"})
    return GDP_df

In [4]:
def answer_three():
    ScimEn_df = pd.read_excel('scimagojr-3.xlsx')
    return ScimEn_df

merge 과정

In [5]:
ScimEn_df = pd.merge(answer_three().iloc[:15,:], answer_one(), how='left', on='Country')
ScimEn_df = pd.merge(ScimEn_df, answer_two(), how='left', on='Country')
ScimEn_df = ScimEn_df.set_index('Rank')
ScimEn_df

,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Rank,,,,,,,,,,,,,,,,,,,,
1,China,127050,126767,597237,411683,4.70,138,123598000000,88,24.392596,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13,1.123328e+13,1.231041e+13,1.389482e+13,1.427994e+13,1.472273e+13
2,United States,96661,94747,792274,265436,8.20,230,90228000000,278,15.806000,1.554258e+13,1.619701e+13,1.678485e+13,1.752716e+13,1.823830e+13,1.874508e+13,1.954298e+13,2.061186e+13,2.143322e+13,2.095303e+13
3,Japan,30504,30287,223024,61554,7.31,134,18116000000,142,16.341609,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12,5.003678e+12,4.930837e+12,5.036892e+12,5.148782e+12,5.057759e+12
4,United Kingdom,20944,20357,206091,37874,9.84,139,7353000000,111,20.788088,2.674891e+12,2.719158e+12,2.803291e+12,3.087166e+12,2.956574e+12,2.722852e+12,2.699017e+12,2.900791e+12,2.878674e+12,2.759804e+12
5,Russian Federation,18534,18301,34266,12422,1.85,57,31182000000,217,17.204246,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12,1.276787e+12,1.574199e+12,1.657329e+12,1.687449e+12,1.483498e+12
6,Canada,17899,17620,215003,40930,12.01,149,12088000000,330,64.585001,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12,1.527995e+12,1.649266e+12,1.725329e+12,1.742015e+12,1.645423e+12
7,Germany,17027,16831,140566,27426,8.26,126,12999000000,158,26.480567,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12,3.469853e+12,3.690849e+12,3.977289e+12,3.888327e+12,3.846414e+12
8,India,15005,14841,128763,37209,8.58,115,38083000000,28,15.726572,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12,2.294798e+12,2.651473e+12,2.701112e+12,2.870504e+12,2.660245e+12
9,France,13153,12973,130632,28601,9.93,114,10278000000,153,16.139551,2.865158e+12,2.683672e+12,2.811877e+12,2.855964e+12,2.439189e+12,2.472964e+12,2.595151e+12,2.790957e+12,2.728870e+12,2.630318e+12


함수로 구현

In [6]:
def answer_four():
    ScimEn_df = pd.merge(answer_three().iloc[:15,:], answer_one(), how='left', on='Country')
    ScimEn_df = pd.merge(ScimEn_df, answer_two(), how='left', on='Country')
    ScimEn_df = ScimEn_df.set_index('Rank')
    return ScimEn_df

In [7]:
answer_four()

,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Rank,,,,,,,,,,,,,,,,,,,,
1,China,127050,126767,597237,411683,4.70,138,123598000000,88,24.392596,7.551500e+12,8.532230e+12,9.570406e+12,1.047568e+13,1.106155e+13,1.123328e+13,1.231041e+13,1.389482e+13,1.427994e+13,1.472273e+13
2,United States,96661,94747,792274,265436,8.20,230,90228000000,278,15.806000,1.554258e+13,1.619701e+13,1.678485e+13,1.752716e+13,1.823830e+13,1.874508e+13,1.954298e+13,2.061186e+13,2.143322e+13,2.095303e+13
3,Japan,30504,30287,223024,61554,7.31,134,18116000000,142,16.341609,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12,5.003678e+12,4.930837e+12,5.036892e+12,5.148782e+12,5.057759e+12
4,United Kingdom,20944,20357,206091,37874,9.84,139,7353000000,111,20.788088,2.674891e+12,2.719158e+12,2.803291e+12,3.087166e+12,2.956574e+12,2.722852e+12,2.699017e+12,2.900791e+12,2.878674e+12,2.759804e+12
5,Russian Federation,18534,18301,34266,12422,1.85,57,31182000000,217,17.204246,2.045926e+12,2.208296e+12,2.292473e+12,2.059242e+12,1.363481e+12,1.276787e+12,1.574199e+12,1.657329e+12,1.687449e+12,1.483498e+12
6,Canada,17899,17620,215003,40930,12.01,149,12088000000,330,64.585001,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12,1.527995e+12,1.649266e+12,1.725329e+12,1.742015e+12,1.645423e+12
7,Germany,17027,16831,140566,27426,8.26,126,12999000000,158,26.480567,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12,3.469853e+12,3.690849e+12,3.977289e+12,3.888327e+12,3.846414e+12
8,India,15005,14841,128763,37209,8.58,115,38083000000,28,15.726572,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12,2.294798e+12,2.651473e+12,2.701112e+12,2.870504e+12,2.660245e+12
9,France,13153,12973,130632,28601,9.93,114,10278000000,153,16.139551,2.865158e+12,2.683672e+12,2.811877e+12,2.855964e+12,2.439189e+12,2.472964e+12,2.595151e+12,2.790957e+12,2.728870e+12,2.630318e+12
